In [1]:
#Libraries for math and data manipulation
import numpy as np
import pandas as pd
import copy
import math
import numpy.random as rand
import scipy as sp
import sys
from datetime import datetime
from collections import Counter

#Dumping stuff
import pickle

#Plotting stuff
import matplotlib.pyplot as plt
import seaborn
import corner
%matplotlib inline
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True) #Want to be able to use tex in the plot labels
seaborn.set_style('ticks')
seaborn.set_color_codes()

#Machine learning stuff
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier   #This is a single decision tree
from sklearn.ensemble import ExtraTreesClassifier #Random forest of trees
from sklearn.linear_model import LogisticRegression #Logistic regression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, precision_score
from sklearn.model_selection import train_test_split

#Table drawing stuff
from texttable import Texttable

#Sci-bayes - add-on for scikit learn that does Bayesian regression.
from skbayes.linear_models import EBLogisticRegression,VBLogisticRegression

In [2]:
def print_array(a, cols, rows):
    if (len(cols) != a.shape[1]) or (len(rows) != a.shape[0]):
        print "Shapes do not match"
        return
    s = a.__repr__()
    s = s.split("array(")[1]
    s = s.replace("      ", "")
    s = s.replace("[[", " [")
    s = s.replace("]])", "]")
    pos = [i for i, ltr in enumerate(s.splitlines()[0]) if ltr == ","]
    pos[-1] = pos[-1]-1
    empty = " " * len(s.splitlines()[0])
    s = s.replace("],", "]")
    s = s.replace(",", "")
    lines = []
    for i, l in enumerate(s.splitlines()):
        lines.append(rows[i] + l)
    s  ="\n".join(lines)
    empty = list(empty)
    for i, p in enumerate(pos):
        empty[p-i] = cols[i]
    s = "".join(empty) + "\n" + s
    print s

In [3]:
def datamap(df,params):
    #Adjust some things that are always included and may need to be used:
    
    #df['dimain'] = df.dimain.map({"Cerebral Malaria":0, "Meningoencephalitis":1, "Meningitis":2.,\
                                       #"Other":3})
    for label in params:
        #Mapping yes/no to 1/0
        if label =='date':
            df['incdate'] = pd.to_datetime(df['incdate'])
            df['date'] = (df['incdate'] - df['incdate'].min())  / np.timedelta64(1,'D')
        if label=='age':
            df['ageym'] = df.ageyrs.astype(float)*12.
            df['agemth'] = df.agemth.astype(float)
            df['age'] = df.ageym + df.agemth #total age in months
            #Mapping yes/no to 1/0
        if label=='clinjaund':
            df[label] = df.clinjaund.map({'Yes': 1, 'No': 0})
        if label=='clinhepato':
            df[label] = df.clinhepato.map({'Yes': 1, 'No': 0})
        if label=='clinspleno':
            df[label] = df.clinspleno.map({'Yes': 1, 'No': 0})
        if label=='clinconv':
            df['clinconv'] = df.clinconv.map({'Yes': 1, 'No': 0})
        if label=='clindehyd':
            df['clindehyd'] = df.clindehyd.map({'Yes': 1, 'No': 0})
        if label=='clinoedem':
            df['clinoedem'] = df.clinoedem.map({'Yes': 1, 'No': 0})
        if label=='clinlymph':
            df['clinlymph'] = df.clinlymph.map({'Yes': 1, 'No': 0})
        if label=='clinresp':
            df['clinresp'] = df.clinresp.map({'Yes': 1, 'No': 0})
        if label=='clinablung':    
            df['clinablung'] = df.clinablung.map({'Yes': 1, 'No': 0})
        if label=='clincyan':
            df['clincyan'] = df.clincyan.map({'Yes': 1, 'No': 0})
        if label=='clincapref':
            df['clincapref'] = df.clincapref.map({'Yes': 1, 'No': 0})
        if label=='clincoldext':
            df['clincoldext'] = df.clincoldext.map({'Yes': 1, 'No': 0})
        if label=='clinorcand':
            df['clinorcand'] = df.clinorcand.map({'Yes': 1, 'No': 0})
        if label=='clinhemmor':
            df['clinhemmor'] = df.clinhemmor.map({'Yes': 1, 'No': 0})
        if label=='clinaids':
            df['clinaids'] = df.clinaids.map({'Yes': 1, 'No': 0})
        if label=='vom':
            df['vom'] = df.vom.map({'Yes': 1, 'No': 0})
        if label=='diar':
            df['diar'] = df.diar.map({'Yes': 1, 'No': 0})
        if label=='headache':
            df['headache'] = df['head'] #Since 'head' is a function
            df['headache'] = df.headache.map({"Don't know":0, 'No':0, 'Yes':1, 'Not applicable':0})
        if label=='conv':
            df['conv'] = df.conv.map({'Yes': 1, 'No': 0})
        if label=='age':
            df['ageym'] = df.ageyrs.astype(float)*12.
            df['agemth'] = df.agemth.astype(float)
            df['age'] = df.ageym + df.agemth #total age in months

        #Medical history variables
        if label=='mhhosp':
            df['mhhosp'] = df.mhhosp.map({'Yes': 1, 'No':0})

        #some slightly more compliated mappings
        if label=='abdpain':
            df['abdpain'] = df.abdpain.map({"Don't know":0, 'No':0, 'Yes':1, 'Not applicable':np.nan})
        if label=='muscle':
            df['muscle'] = df.muscle.map({"Don't know":0, 'No':0, 'Yes':1, 'Not applicable':np.nan})
        if label=='mhdevsp':
            df['mhdevsp'] = df.mhdevsp.map({'OTHER':1, 'GLOBAL DEVELOPMENTAL DELAY':1, 'HYDROCEPHALUS':1,
                   'HEARING LOSSES':1, 'MOTOR DEVELOPEMENTAL DELAY':1,
                   'SPEECH DEVELOPEMENTAL DELAY':1})
        #malaria rapid disagnositc
        if label=='malaria':
            df['malaria'] = df.malaria.map({'confirmed':2, 'no':0, 'probable':1})
            
        #outcome
        if label=='d3evol':
            df['d3evol'] = df.d3evol.map({'Gradual worsening':-1, 'Unchanged':0, 'Improved':1,\
                                          'Improved & worsened':-1})

        #Initial diagnosis coding: (either 'diagnosed - 0', or 'undiagnosed - 1')
        if label=='CMalaria':
            df['CMalaria'] = df.dimain.map({0:1,1:0,2:0,3:0})
        if label=='Enceph':
            df['Enceph'] = df.dimain.map({0:0,1:1,2:0,3:0})
        if label=='Mening':
            df['Mening'] = df.dimain.map({0:0,1:0,2:1,3:0})
        if label=='Other':
            df['Other'] = df.dimain.map({0:0,1:0,2:0,3:1})


        #This coding is going to be important. You'll want to figure out which categories are useful, and which
        #are practical.
        if label=='diag':
            df['diag'] = df.diag.map({'malaria':0, 'cereb malaria':0,'virus-malaria':0,\
                                                'virus-bacteria':0, 'bacteremia':0,'bact meningitis':0,\
                                                'virus-other':0,'virus':0,'malaria-bacterial':0,\
                                                'tb':0,'crypto':0, '0.0':1})
        if label=='ttatb':
            df['ttatb'] = df.ttatb.map({'No':0,'Yes':1})
        if label=='ttmal':
            df['ttmal'] = df.ttmal.map({'No':0,'Yes':2})
        

In [63]:
def trainVBLR(features,data,vblr,modelA,depth=0):
    features = np.array(features)
    Npar = len(features) - 1 #dimain and diag are always included in the list, but aren't model params
    XY_df = data.loc[:,features].dropna().copy()
    
    Input = XY_df.values
    X = Input[:,0:-1].copy()
    Y = Input[:,-1].copy()
    Y = Y.astype(float)
        
    #Map the three different models into model A vs. Other: 
    if modelA==1:    #antibiotics vs. other
        Y[Y==3.] = 2.
    if modelA==2:    #antimalarials vs. other
        Y[Y==1.] = 3.
    if modelA==3:    #both vs. other
        Y[Y==2.] = 1.
 
    
    print 'Number of patients in data set:', len(Y)
    print 'Number of features:', len(features)-1
    print 'Fitting model ...'
        
    vblr.__init__()
    vblr.fit(X,Y)    
    if depth<=1:  #If you're only fitting once
        return Input, features
        print 'model complete!'
    
    if depth>1:        #go through 'depth' number of times and drop all coefficients consistent with zero.
        depth -= 1
        covar_mat = copy.copy(vblr.sigma_)
        covar_mat = np.array(covar_mat)
        covar_mat = covar_mat.reshape([Npar+1,Npar+1])
        post_peaks = np.zeros([Npar])
        sigmas = np.zeros([Npar])
        for i in range(Npar):
            cval = vblr.coef_[0][i]
            post_peaks[i] = cval
            sigmas[i] = np.sqrt(covar_mat[i+1,i+1])

            post_peaks = np.array(post_peaks)  
            sigmas = np.array(sigmas)

            #Which are consistent (within 1 sigma) with zero?
            post_plus = post_peaks + 0.75*sigmas
            post_minus = post_peaks - 0.75*sigmas

            splus = np.sign(post_plus)
            sminus = np.sign(post_minus)

        keep = splus==sminus
        extra = [True]   #Need to hang onto the category label
        keep = np.concatenate([keep,extra])
        features = features[keep]
        print 'Non-zero Coefficients:',len(features)-1
        return trainVBLR(features,data,vblr,modelA,depth)
        depth = 0


In [64]:
#Read in the raw data file:
data_raw = pd.read_csv("../Data/cns_data.csv")   #Using Pandas for data manipulation
data_df = data_raw.drop(['surnamenam','patinit','patientreg','mothinit'],axis=1)

#Split up 'incdate' into month, day, and year. Monthday is meant to capture seasonality.
data_df['month'] = pd.DatetimeIndex(data_df['incdate']).month
data_df['day'] = pd.DatetimeIndex(data_df['incdate']).day
data_df['year'] = pd.DatetimeIndex(data_df['incdate']).year
data_df['monthday'] = data_df['month']*30+data_df['day']

In [65]:
#Select the patients who have improved. We are using this as an indicator of the 'right' treatment, although
#we know that this has limitations.
patients=data_df[data_df.d3evol=='Improved'].copy()
print 'There are', len(patients), 'patients who improved.'

#What parameters do we want to consider?
pmap = ['glasgtot','clinjaund','clinhepato','clinspleno','clinconv','clindehyd',\
                          'clinoedem','clinlymph','clinresp','clinablung','clincyan','clincapref','clincoldext',\
                         'clinorcand','clinhemmor','clinaids',\
                         'abdpain','vom','diar','headache','muscle','conv',\
                         'age','date',\
                         'temp','card','resp','sbp','dbp','weight','height',\
                         'malaria',
                         'mhhosp',\
               'ttatb','ttmal'] #These will be used to define the final treatment.

#Map all of these parameters to numerical values:
datamap(patients,pmap)
#Add the final category to the data
patients['FinalTreatment'] = patients.ttmal+patients.ttatb #0 - none, 1 - atb, 2 - atm, 3 - both


There are 328 patients who improved.


In [69]:
#Train the vblr 
feature_list = ['glasgtot','clinjaund','clinhepato','clinspleno','clinconv','clindehyd',\
                          'clinoedem','clinlymph','clinresp','clinablung','clincyan','clincapref','clincoldext',\
                         'clinorcand','clinhemmor','clinaids',\
                         'abdpain','vom','diar','headache','muscle','conv',\
                         'age','date',\
                         'temp','card','resp','sbp','dbp','weight','height',\
                          'mhhosp', \
               'FinalTreatment']
vblr_both = VBLogisticRegression()
DataTrained, feat_kept_both = (trainVBLR(feature_list,patients,vblr_both,modelA=1,depth=6))
#print 'There are', len(feat_kept)-1, 'features in this model.'
#print 'These features are:', feat_kept

Number of patients in data set: 180
Number of features: 32
Fitting model ...
nf 32 nsig 33
Non-zero Coefficients: 16
Number of patients in data set: 180
Number of features: 16
Fitting model ...
nf 16 nsig 17
Non-zero Coefficients: 16
Number of patients in data set: 180
Number of features: 16
Fitting model ...
nf 16 nsig 17
Non-zero Coefficients: 16
Number of patients in data set: 180
Number of features: 16
Fitting model ...
nf 16 nsig 17
Non-zero Coefficients: 16
Number of patients in data set: 180
Number of features: 16
Fitting model ...
nf 16 nsig 17
Non-zero Coefficients: 16
Number of patients in data set: 180
Number of features: 16
Fitting model ...
nf 16 nsig 17


In [32]:
#Dump the three models and the three feature lists to a pickle:
"""
outlist = [vblr_atb,feat_kept_atb,vblr_atm,feat_kept_atm,vblr_both,feat_kept_both]
PIK = "three_model_pickle_nomal_test.p"
with open(PIK, "wb") as f:
    pickle.dump(outlist, f)
"""   

'\noutlist = [vblr_atb,feat_kept_atb,vblr_atm,feat_kept_atm,vblr_both,feat_kept_both]\nPIK = "three_model_pickle_nomal_test.p"\nwith open(PIK, "wb") as f:\n    pickle.dump(outlist, f)\n'

In [39]:
#The above was all about training the model to generate the 'machine learning' policy.
#Generate the machine learning policy action for each patient:

#Load models and feature lists
modelList = pickle.load( open( "three_model_pickle.p", "rb" ) )

pantb_model = modelList[0]
pantm_model = modelList[2]
pboth_model = modelList[4]

featAntb = list(modelList[1][:-1])
featAntm = list(modelList[3][:-1])
featBoth = list(modelList[5][:-1])

allFeat = featAntb + featAntm + featBoth
allFeat = list(set(allFeat))    #Collection of all of the features used - will want to test only on patients with
                                #non-NaN values for all of these.
allFeat.append('atatb')
allFeat.append('atmal')
allFeat.append('FinalTreatment')

finalPatients_df = patients[allFeat].copy()
finalPatients_df['atmal'] = finalPatients_df.atmal.map({'No':0,'Yes':1, np.nan:0})*2
finalPatients_df['atatb'] = finalPatients_df.atatb.map({'No':0,'Yes':1, 'Not applicable':0})
finalPatients_df = finalPatients_df.dropna()  #Only keep patients with values for all of these features.
finalPatients = finalPatients_df.values

Npatient = len(finalPatients)

print 'There are', Npatient, 'patients...'
print 'and', len(allFeat)-1, 'features.'

#Now, you need to go through and calculate the probability of each of the three treatments for each patient, and
#assign that patient to the category with the highest probability. 
#From the list of patients who have values for all possible features, select the features that were used in 
#each model:
atbFeatures = finalPatients_df[featAntb].values
atmFeatures = finalPatients_df[featAntm].values
bothFeatures = finalPatients_df[featBoth].values

TrueCat = finalPatients[:,-1]

#Generate the probabilities for each model:
pantb = pantb_model.predict_proba(atbFeatures)
pantm = pantm_model.predict_proba(atmFeatures)
pboth = pboth_model.predict_proba(bothFeatures)

Policy_ML = []
for i in range(Npatient):
    pvec = np.array([pantb[i,0]*100,pantm[i,0]*100,pboth[i,0]*100])
    Policy_ML.append(np.where(pvec==np.max(pvec))[0][0]+1)
Policy_ML = np.array(Policy_ML)
Policy_ML = Policy_ML.astype(float)

print allFeat

There are 180 patients...
and 27 features.
['clinhepato', 'conv', 'vom', 'resp', 'height', 'dbp', 'clinresp', 'weight', 'clinablung', 'clinconv', 'clinspleno', 'diar', 'headache', 'abdpain', 'malaria', 'card', 'clinlymph', 'clinoedem', 'glasgtot', 'temp', 'age', 'clinjaund', 'mhhosp', 'clindehyd', 'muscle', 'atatb', 'atmal', 'FinalTreatment']


In [40]:
#Same method, but malaria test not included

#Load models and feature lists
modelList = pickle.load( open( "three_model_pickle_nomal_OLD.p", "rb" ) )

pantb_model = modelList[0]
pantm_model = modelList[2]
pboth_model = modelList[4]

featAntb = list(modelList[1][:-1])
featAntm = list(modelList[3][:-1])
featBoth = list(modelList[5][:-1])

allFeat = featAntb + featAntm + featBoth
allFeat = list(set(allFeat))    #Collection of all of the features used - will want to test only on patients with
                                #non-NaN values for all of these.
allFeat.append('atatb')
allFeat.append('atmal')
allFeat.append('FinalTreatment')

finalPatients_df = patients[allFeat].copy()
finalPatients_df['atmal'] = finalPatients_df.atmal.map({'No':0,'Yes':1, np.nan:0})*2
finalPatients_df['atatb'] = finalPatients_df.atatb.map({'No':0,'Yes':1, 'Not applicable':0})
finalPatients_df = finalPatients_df.dropna()  #Only keep patients with values for all of these features.
finalPatients = finalPatients_df.values

Npatient = len(finalPatients)

print 'There are', Npatient, 'patients...'
print 'and', len(allFeat)-1, 'features.'

#Now, you need to go through and calculate the probability of each of the three treatments for each patient, and
#assign that patient to the category with the highest probability. 
#From the list of patients who have values for all possible features, select the features that were used in 
#each model:
atbFeatures = finalPatients_df[featAntb].values
atmFeatures = finalPatients_df[featAntm].values
bothFeatures = finalPatients_df[featBoth].values

TrueCat = finalPatients[:,-1]

#Generate the probabilities for each model:
pantb = pantb_model.predict_proba(atbFeatures)
pantm = pantm_model.predict_proba(atmFeatures)
pboth = pboth_model.predict_proba(bothFeatures)

Policy_ML_nomal = []
for i in range(Npatient):
    pvec = np.array([pantb[i,0]*100,pantm[i,0]*100,pboth[i,0]*100])
    Policy_ML_nomal.append(np.where(pvec==np.max(pvec))[0][0]+1)
Policy_ML_nomal = np.array(Policy_ML_nomal)
Policy_ML_nomal = Policy_ML_nomal.astype(float)

print allFeat

There are 180 patients...
and 27 features.
['clinhepato', 'conv', 'vom', 'resp', 'height', 'dbp', 'clinresp', 'weight', 'clinhemmor', 'clinablung', 'clinconv', 'clinspleno', 'diar', 'headache', 'abdpain', 'date', 'clinlymph', 'clinoedem', 'glasgtot', 'temp', 'age', 'clinjaund', 'mhhosp', 'clindehyd', 'muscle', 'atatb', 'atmal', 'FinalTreatment']


In [41]:
#Intake policy is just what was done directly on intake
Policy_In = finalPatients_df['atmal'] + finalPatients_df['atatb']
Policy_In = Policy_In.values
Policy_In = Policy_In.astype(float)
Policy_In[Policy_In==0.] = 4.

In [42]:
#Everyone gets everything policy is just what it sounds like
Policy_EgE = np.ones(Npatient)*3.

In [43]:
#And try a random policy just for grins:
poss = [1,2,3,4]
Policy_Rand = np.random.choice(poss,size=[Npatient])

In [44]:
#What everyone *should* have gotten
Truth = finalPatients_df['FinalTreatment'].values
Truth = Truth.astype(float)

print np.sum(Policy_ML==Truth)

120


In [45]:
#Now, go through and fill out the 'policy grid' for each of these three policies.
Pij_ML = np.zeros([4,4])
Pij_ML_nm = np.zeros([4,4])
Pij_I = np.zeros([4,4])
Pij_EgE = np.zeros([4,4])
Pij_R = np.zeros([4,4])

poss = [1,2,3,4]

for i in poss:
    for j in poss:
        needmask = Truth==i
        
        gotmask = Policy_ML==j 
        Pij_ML[i-1,j-1] = sum(needmask*gotmask)
        
        gotmask = Policy_ML_nomal==j
        Pij_ML_nm[i-1,j-1] = sum(needmask*gotmask)
        
        gotmask = Policy_In==j
        Pij_I[i-1,j-1] = sum(needmask*gotmask)
        
        gotmask = Policy_EgE==j
        Pij_EgE[i-1,j-1] = sum(needmask*gotmask)
        
        gotmask = Policy_Rand==j
        Pij_R[i-1,j-1] = sum(needmask*gotmask)
        

In [46]:
c = ["ab","am","b","n"]
r = ["atb ","atm ","both","none"]

print "Machine Learning Policy:"
print_array(Pij_ML,c,r)
print "\n"

print "Machine Learning Policy - no RMD:"
print_array(Pij_ML_nm,c,r)
print "\n"

print "Intake Policy:"
print_array(Pij_I,c,r)
print "\n"

print "Random Policy:"
print_array(Pij_R,c,r)
print "\n"

print "Everyone gets Everything:"
print_array(Pij_EgE,c,r)

Machine Learning Policy:
     ab   am   b   n    
atb  [35.  0. 16.  0.]
atm  [ 0. 12.  8.  0.]
both [16. 20. 73.  0.]
none [ 0.  0.  0.  0.]


Machine Learning Policy - no RMD:
     ab   am   b   n    
atb  [22.  1. 28.  0.]
atm  [ 0.  7. 13.  0.]
both [11. 14. 84.  0.]
none [ 0.  0.  0.  0.]


Intake Policy:
     ab   am   b   n    
atb  [21.  1.  1. 28.]
atm  [ 0. 11.  0.  9.]
both [ 7. 19. 57. 26.]
none [ 0.  0.  0.  0.]


Random Policy:
     ab   am   b   n    
atb  [11. 12. 14. 14.]
atm  [ 1. 11.  4.  4.]
both [28. 25. 26. 30.]
none [ 0.  0.  0.  0.]


Everyone gets Everything:
      ab    am    b    n    
atb  [  0.   0.  51.   0.]
atm  [  0.   0.  20.   0.]
both [  0.   0. 109.   0.]
none [  0.   0.   0.   0.]


In [47]:
#Fill in the returns matrix. This should be interactive somehow.
Rij = [[1.,-2.,0.5,-2.],[-2.,1.,0.5,-2.],[-2.,-2.,1.,-2.],[0.5,0.5,0.,1.]]

In [72]:
print 'Random:', np.sum(Pij_R*Rij)/Npatient
print 'ML:', np.sum(Pij_ML*Rij)/Npatient
print 'ML - no MRD:', np.sum(Pij_ML_nm*Rij)/Npatient
print 'Intake:', np.sum(Pij_I*Rij)/Npatient
print 'EgE:', np.sum(Pij_EgE*Rij)/Npatient

Random: -0.95
ML: 0.3333333333333333
ML - no MRD: 0.4527777777777778
Intake: -0.5027777777777778
EgE: 0.8027777777777778


In [ ]:
#NEXT: time-discounting. What happens if you give people the *exact* right thing - but several days later? 